In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K

from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *

#Get data 
cwd = os.path.normpath(os.getcwd())
df = pd.read_csv(cwd+'/data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)

df_array = []
for idx in range(30):
    df_array.append(df[[f'User{idx+1}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx+1}_lag_24hrs']])

#df_array[3].head(3)

#Train, Validation and Test datasets
sequence_length = 25
batch_size = 16
num_features = df_array[0].shape[1]

dh = Datahandler()

X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = dh.min_max_scaling(train_df)
    val_df = dh.min_max_scaling(val_df)
    test_df = dh.min_max_scaling(test_df)

    # Sequencing
    train_sequences = dh.create_sequences(train_df, sequence_length)
    val_sequences = dh.create_sequences(val_df, sequence_length)
    test_sequences = dh.create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_train[f'user{idx+1}'], y_train[f'user{idx+1}'] = dh.prepare_data(train_sequences, batch_size)
    X_val[f'user{idx+1}'], y_val[f'user{idx+1}'] = dh.prepare_data(val_sequences, batch_size)
    X_test[f'user{idx+1}'], y_test[f'user{idx+1}'] = dh.prepare_data(test_sequences, batch_size)

#General Hyperparameters
# #All models
horizon = 1
max_epochs = 100
m1 = ModelGenerator()
mh = Modelhandler()

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()
#model_checkpoint = ModelCheckpoint('models/best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
callbacks=[early_stopping, timing_callback, custom_callback] #model_checkpoint


In [2]:
def avg_weights_with_noise_fedprox(weight_list, clip_threshold=None, noise_scale=0.001, proximal_term=0.1):
    avg_grad = list()

    for grad_list_tuple in zip(*weight_list):
        layer_mean = tf.math.reduce_mean(grad_list_tuple, axis=0)

        if clip_threshold is not None:
            layer_mean = tf.clip_by_value(layer_mean, -clip_threshold, clip_threshold)

        noise = tf.random.normal(shape=layer_mean.shape, mean=0.0, stddev=noise_scale)
        noisy_layer_mean = layer_mean + noise

        # Add FedProx proximal term
        proximal_update = -proximal_term * noisy_layer_mean

        avg_grad.append(noisy_layer_mean + proximal_update)

    return avg_grad

# Federated Learning Dense MoE Model

In [3]:
dense_results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

In [4]:
num_clusters = 4
y = np.loadtxt(f'evaluations/federated_learning/clusters_KMeans{num_clusters}_dtw.csv', delimiter=',').astype(int)

cluster_users = {i: [] for i in range(num_clusters)}

# Iterate through each cluster
for cluster_number in range(num_clusters):
    users_in_cluster = np.where(y == cluster_number)[0] +1
    cluster_users[cluster_number] = users_in_cluster
cluster_users

{0: array([ 1,  2,  4,  5,  7, 10, 11, 12, 15, 16, 17, 21, 22, 23, 24, 25, 28,
        30], dtype=int64),
 1: array([29], dtype=int64),
 2: array([ 6, 13, 18, 20, 26], dtype=int64),
 3: array([ 3,  8,  9, 14, 19, 27], dtype=int64)}

In [5]:
#Dense MODEL 1 ------------------------------------------------------------------
dense_moe_architecture = "Dense_MoE_e4d8_d16"
dense_units = 16
num_experts = 4
expert_units = 8

# Create global models for each cluser (6)
for cluster in range(num_clusters):
#Build and save global model
    global_dense_moe = m1.build_soft_dense_moe_model(X_train[f'user{1}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster}/{dense_moe_architecture}/FederatedRound{0}")

  
federated_rounds = 3
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    for cluster_number, users_in_cluster in cluster_users.items():
        print(f"Cluster {cluster_number}:")

        #Get global models weights
        global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{dense_moe_architecture}/FederatedRound{federated_round}", compile=False)
        global_dense_moe_weights = global_dense_moe.get_weights()

        #initial list for local model weights
        local_dense_moe_weight_list = list()


        #for idx, user in enumerate(df_array): 
        for user_index in users_in_cluster:
            user_df = df_array[user_index-1]  # Get the user's DataFrame from the array
            print(f"User {user_index}") 
                      
            #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
            local_dense_moe_model = m1.build_soft_dense_moe_model(X_train[f'user{user_index}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
            local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), metrics=metrics)

            #set local model weight to the weight of the global model
            local_dense_moe_model.set_weights(global_dense_moe_weights)
            
            #Fit local model to local data
            dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
                model=local_dense_moe_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = max_epochs, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=dense_moe_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
            )
            #add model weights to list        
            local_dense_moe_weights = local_dense_moe_model.get_weights()
            local_dense_moe_weight_list.append(local_dense_moe_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights_dense_moe = avg_weights_with_noise_fedprox(local_dense_moe_weight_list)
        #update global model 
        global_dense_moe.set_weights(average_weights_dense_moe)
        #Save global models
        global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{dense_moe_architecture}/FederatedRound{federated_round+1}")
        print("Saved Global models")


#Evaluation
dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

for cluster_number, users_in_cluster in cluster_users.items():
    print(f"Cluster {cluster_number}:")

    #Get global models weights
    global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{dense_moe_architecture}/FederatedRound{federated_rounds}", compile=False)

    #for idx, user in enumerate(df_array): 
    for user_index in users_in_cluster:
        print("User: ", user_index)
        for round in range(3):
            global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{dense_moe_architecture}/FederatedRound{federated_rounds}", compile=False)
            dense_moe_model = m1.build_soft_dense_moe_model(X_train[f'user{user_index}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
            dense_moe_model.set_weights(global_dense_moe.get_weights())
            
            dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
                model=dense_moe_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = 1, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=dense_moe_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006)
            )
            # Add the 'architecture' column from dense_user_results to dense_results
            dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')  

for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_moe_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row



INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/Dense_MoE_e4d8_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/Dense_MoE_e4d8_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/Dense_MoE_e4d8_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/Dense_MoE_e4d8_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/Dense_MoE_e4d8_d16/FederatedRound0\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/Dense_MoE_e4d8_d16/FederatedRound3\assets


Saved Global models
Cluster 3:
User 3
User 8
User 9
User 14
User 19
User 27


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/Dense_MoE_e4d8_d16/FederatedRound3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/Dense_MoE_e4d8_d16/FederatedRound3\assets


Saved Global models
Cluster 0:
User:  1
User:  2
User:  4
User:  5
User:  7
User:  10
User:  11
User:  12
User:  15
User:  16
User:  17
User:  21
User:  22
User:  23
User:  24
User:  25
User:  28
User:  30
Cluster 1:
User:  29
Cluster 2:
User:  6
User:  13
User:  18
User:  20
User:  26
Cluster 3:
User:  3
User:  8
User:  9
User:  14
User:  19
User:  27


In [6]:
dense_results.to_csv(f'evaluations/clustered_federated_learning/{dense_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_MoE_e4d8_d16,1.495996,1.413643,0.034502,0.000277,0.185745,0.000746,9.506578e+04,10724.872810,0.132685,0.001910
1,Dense_MoE_e4d8_d16,1.350153,1.270475,0.017793,0.000942,0.133360,0.003506,2.148979e+05,31327.194595,0.074563,0.001257
2,Dense_MoE_e4d8_d16,1.374985,1.293299,0.019011,0.000143,0.137878,0.000519,5.378165e+04,5117.348585,0.073185,0.004528
3,Dense_MoE_e4d8_d16,1.375389,1.294036,0.021595,0.000361,0.146948,0.001226,1.556666e+05,11608.861413,0.063102,0.001889
4,Dense_MoE_e4d8_d16,1.382867,1.301181,0.013548,0.000742,0.116367,0.003170,7.609251e+04,9830.769647,0.070237,0.001908
5,Dense_MoE_e4d8_d16,1.398616,1.314584,0.018657,0.000397,0.136585,0.001459,8.944397e+04,6192.629211,0.096862,0.001518
6,Dense_MoE_e4d8_d16,1.547030,1.465073,0.018713,0.000332,0.136792,0.001218,7.546588e+04,3156.671672,0.078188,0.000688
7,Dense_MoE_e4d8_d16,1.358829,1.277476,0.025183,0.001245,0.158659,0.003950,2.673365e+04,2865.039978,0.099982,0.001196
8,Dense_MoE_e4d8_d16,1.362834,1.281814,0.023523,0.000862,0.153356,0.002805,6.924008e+01,1.259674,0.089070,0.001563
9,Dense_MoE_e4d8_d16,1.370121,1.289102,0.013432,0.000491,0.115884,0.002114,2.110753e+06,208456.112380,0.066890,0.001273


In [7]:
#Dense MODEL 1 ------------------------------------------------------------------
dense_moe_architecture = "TopK_Dense_MoE" #top2_exp5_d8
dense_units = 16

num_experts = 5
expert_units = 8
top_k = 2

# Create global models for each cluser (6)
for cluster in range(num_clusters):
#Build and save global model
    global_dense_moe = m1.build_topk_dense_moe_model(X_train[f'user{1}'], batch_size, horizon, dense_units, num_experts, top_k, expert_units, m1)
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster}/{dense_moe_architecture}/FederatedRound{0}")

  
federated_rounds = 3
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    for cluster_number, users_in_cluster in cluster_users.items():
        print(f"Cluster {cluster_number}:")

        #Get global models weights
        global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{dense_moe_architecture}/FederatedRound{federated_round}", compile=False)
        global_dense_moe_weights = global_dense_moe.get_weights()

        #initial list for local model weights
        local_dense_moe_weight_list = list()


        #for idx, user in enumerate(df_array): 
        for user_index in users_in_cluster:
            user_df = df_array[user_index-1]  # Get the user's DataFrame from the array
            print(f"User {user_index}") 
                      
            #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
            local_dense_moe_model = m1.build_topk_dense_moe_model(X_train[f'user{user_index}'], batch_size, horizon, dense_units, num_experts, top_k, expert_units, m1)
            local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), metrics=metrics)

            #set local model weight to the weight of the global model
            local_dense_moe_model.set_weights(global_dense_moe_weights)
            
            #Fit local model to local data
            dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
                model=local_dense_moe_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = max_epochs, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=dense_moe_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
            )
            #add model weights to list        
            local_dense_moe_weights = local_dense_moe_model.get_weights()
            local_dense_moe_weight_list.append(local_dense_moe_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights_dense_moe = avg_weights_with_noise_fedprox(local_dense_moe_weight_list)
        #update global model 
        global_dense_moe.set_weights(average_weights_dense_moe)
        #Save global models
        global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{dense_moe_architecture}/FederatedRound{federated_round+1}")
        print("Saved Global models")


#Evaluation
dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

for cluster_number, users_in_cluster in cluster_users.items():
    print(f"Cluster {cluster_number}:")

    #Get global models weights
    global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{dense_moe_architecture}/FederatedRound{federated_rounds}", compile=False)

    #for idx, user in enumerate(df_array): 
    for user_index in users_in_cluster:
        print("User: ", user_index)
        for round in range(3):
            global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{dense_moe_architecture}/FederatedRound{federated_rounds}", compile=False)
            dense_moe_model = m1.build_topk_dense_moe_model(X_train[f'user{user_index}'], batch_size, horizon, dense_units, num_experts, top_k, expert_units, m1)
            dense_moe_model.set_weights(global_dense_moe.get_weights())
            
            dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
                model=dense_moe_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = 1, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=dense_moe_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006)
            )
            # Add the 'architecture' column from dense_user_results to dense_results
            dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')  

for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_moe_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row



INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/TopK_Dense_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/TopK_Dense_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/TopK_Dense_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/TopK_Dense_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/TopK_Dense_MoE/FederatedRound0\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/TopK_Dense_MoE/FederatedRound3\assets


Saved Global models
Cluster 3:
User 3
User 8
User 9
User 14
User 19
User 27


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/TopK_Dense_MoE/FederatedRound3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/TopK_Dense_MoE/FederatedRound3\assets


Saved Global models
Cluster 0:
User:  1
User:  2
User:  4
User:  5
User:  7
User:  10
User:  11
User:  12
User:  15
User:  16
User:  17
User:  21
User:  22
User:  23
User:  24
User:  25
User:  28
User:  30
Cluster 1:
User:  29
Cluster 2:
User:  6
User:  13
User:  18
User:  20
User:  26
Cluster 3:
User:  3
User:  8
User:  9
User:  14
User:  19
User:  27


In [8]:
dense_results.to_csv(f'evaluations/clustered_federated_learning/{dense_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_MoE_e4d8_d16,1.495996,1.413643,0.034502,0.000277,0.185745,0.000746,9.506578e+04,10724.872810,0.132685,0.001910
1,Dense_MoE_e4d8_d16,1.350153,1.270475,0.017793,0.000942,0.133360,0.003506,2.148979e+05,31327.194595,0.074563,0.001257
2,Dense_MoE_e4d8_d16,1.374985,1.293299,0.019011,0.000143,0.137878,0.000519,5.378165e+04,5117.348585,0.073185,0.004528
3,Dense_MoE_e4d8_d16,1.375389,1.294036,0.021595,0.000361,0.146948,0.001226,1.556666e+05,11608.861413,0.063102,0.001889
4,Dense_MoE_e4d8_d16,1.382867,1.301181,0.013548,0.000742,0.116367,0.003170,7.609251e+04,9830.769647,0.070237,0.001908
5,Dense_MoE_e4d8_d16,1.398616,1.314584,0.018657,0.000397,0.136585,0.001459,8.944397e+04,6192.629211,0.096862,0.001518
6,Dense_MoE_e4d8_d16,1.547030,1.465073,0.018713,0.000332,0.136792,0.001218,7.546588e+04,3156.671672,0.078188,0.000688
7,Dense_MoE_e4d8_d16,1.358829,1.277476,0.025183,0.001245,0.158659,0.003950,2.673365e+04,2865.039978,0.099982,0.001196
8,Dense_MoE_e4d8_d16,1.362834,1.281814,0.023523,0.000862,0.153356,0.002805,6.924008e+01,1.259674,0.089070,0.001563
9,Dense_MoE_e4d8_d16,1.370121,1.289102,0.013432,0.000491,0.115884,0.002114,2.110753e+06,208456.112380,0.066890,0.001273


# Federated Learning Bidirectional LSTM Moe

In [9]:
#Dense MODEL 1 ------------------------------------------------------------------
bilstm_moe_architecture = "Soft_BiLSTM_MoE"
lstm_units = 4
num_experts = 4
expert_units = 8

# Create global models for each cluser (6)
for cluster in range(num_clusters):
#Build and save global model
    global_dense_moe = m1.build_soft_biLSTM_moe_model(X_train[f'user{1}'], batch_size, horizon, lstm_units, num_experts, expert_units, m1)
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster}/{bilstm_moe_architecture}/FederatedRound{0}")

  
federated_rounds = 3
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    for cluster_number, users_in_cluster in cluster_users.items():
        print(f"Cluster {cluster_number}:")

        #Get global models weights
        global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{bilstm_moe_architecture}/FederatedRound{federated_round}", compile=False)
        global_dense_moe_weights = global_dense_moe.get_weights()

        #initial list for local model weights
        local_dense_moe_weight_list = list()


        #for idx, user in enumerate(df_array): 
        for user_index in users_in_cluster:
            user_df = df_array[user_index-1]  # Get the user's DataFrame from the array
            print(f"User {user_index}") 
                      
            #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
            local_dense_moe_model = m1.build_soft_biLSTM_moe_model(X_train[f'user{user_index}'], batch_size, horizon, lstm_units, num_experts, expert_units, m1)
            local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), metrics=metrics)

            #set local model weight to the weight of the global model
            local_dense_moe_model.set_weights(global_dense_moe_weights)
            
            #Fit local model to local data
            dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
                model=local_dense_moe_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = max_epochs, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=bilstm_moe_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
            )
            #add model weights to list        
            local_dense_moe_weights = local_dense_moe_model.get_weights()
            local_dense_moe_weight_list.append(local_dense_moe_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights_dense_moe = avg_weights_with_noise_fedprox(local_dense_moe_weight_list)
        #update global model 
        global_dense_moe.set_weights(average_weights_dense_moe)
        #Save global models
        global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{bilstm_moe_architecture}/FederatedRound{federated_round+1}")
        print("Saved Global models")


#Evaluation
dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

for cluster_number, users_in_cluster in cluster_users.items():
    print(f"Cluster {cluster_number}:")

    #Get global models weights
    global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{bilstm_moe_architecture}/FederatedRound{federated_rounds}", compile=False)

    #for idx, user in enumerate(df_array): 
    for user_index in users_in_cluster:
        print("User: ", user_index)
        for round in range(3):
            global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{bilstm_moe_architecture}/FederatedRound{federated_rounds}", compile=False)
            dense_moe_model = m1.build_soft_biLSTM_moe_model(X_train[f'user{user_index}'], batch_size, horizon, lstm_units, num_experts, expert_units, m1)
            dense_moe_model.set_weights(global_dense_moe.get_weights())
            
            dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
                model=dense_moe_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = 1, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=bilstm_moe_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006)
            )
            # Add the 'architecture' column from dense_user_results to dense_results
            dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')  

for idx in range(len(df_array)):
    new_row = {
        'architecture': bilstm_moe_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/Soft_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/Soft_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/Soft_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/Soft_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/Soft_BiLSTM_MoE/FederatedRound0\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/Soft_BiLSTM_MoE/FederatedRound3\assets


Saved Global models
Cluster 3:
User 3
User 8
User 9
User 14
User 19
User 27


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/Soft_BiLSTM_MoE/FederatedRound3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/Soft_BiLSTM_MoE/FederatedRound3\assets


Saved Global models
Cluster 0:
User:  1
User:  2
User:  4
User:  5
User:  7
User:  10
User:  11
User:  12
User:  15
User:  16
User:  17
User:  21
User:  22
User:  23
User:  24
User:  25
User:  28
User:  30
Cluster 1:
User:  29
Cluster 2:
User:  6
User:  13
User:  18
User:  20
User:  26
Cluster 3:
User:  3
User:  8
User:  9
User:  14
User:  19
User:  27


In [10]:
dense_results.to_csv(f'evaluations/clustered_federated_learning/{bilstm_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_MoE_e4d8_d16,1.495996,1.413643,0.034502,0.000277,0.185745,0.000746,9.506578e+04,10724.872810,0.132685,0.001910
1,Dense_MoE_e4d8_d16,1.350153,1.270475,0.017793,0.000942,0.133360,0.003506,2.148979e+05,31327.194595,0.074563,0.001257
2,Dense_MoE_e4d8_d16,1.374985,1.293299,0.019011,0.000143,0.137878,0.000519,5.378165e+04,5117.348585,0.073185,0.004528
3,Dense_MoE_e4d8_d16,1.375389,1.294036,0.021595,0.000361,0.146948,0.001226,1.556666e+05,11608.861413,0.063102,0.001889
4,Dense_MoE_e4d8_d16,1.382867,1.301181,0.013548,0.000742,0.116367,0.003170,7.609251e+04,9830.769647,0.070237,0.001908
...,...,...,...,...,...,...,...,...,...,...,...
85,Soft_BiLSTM_MoE,4.052393,3.921894,0.021977,0.000236,0.148244,0.000798,9.824401e+04,23268.604926,0.088703,0.000406
86,Soft_BiLSTM_MoE,4.356113,4.225749,0.016887,0.000127,0.129950,0.000487,1.760885e+05,6127.874173,0.088117,0.001665
87,Soft_BiLSTM_MoE,3.948699,3.820669,0.010527,0.000138,0.102599,0.000672,4.613215e+04,7909.502836,0.043659,0.000603
88,Soft_BiLSTM_MoE,4.225190,4.100161,0.017087,0.000407,0.130713,0.001553,4.136983e+05,34033.598254,0.093070,0.000742


In [11]:
#Dense MODEL 1 ------------------------------------------------------------------
bilstm_moe_architecture = "TopK_BiLSTM_MoE"
lstm_units = 8

num_experts = 4
top_k = 2
expert_units = 8

# Create global models for each cluser (6)
for cluster in range(num_clusters):
#Build and save global model
    global_dense_moe = m1.build_topk_bilstm_moe_model(X_train[f'user{1}'], batch_size, horizon, lstm_units, num_experts, top_k, expert_units, m1)
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster}/{bilstm_moe_architecture}/FederatedRound{0}")

  
federated_rounds = 3
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    for cluster_number, users_in_cluster in cluster_users.items():
        print(f"Cluster {cluster_number}:")

        #Get global models weights
        global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{bilstm_moe_architecture}/FederatedRound{federated_round}", compile=False)
        global_dense_moe_weights = global_dense_moe.get_weights()

        #initial list for local model weights
        local_dense_moe_weight_list = list()


        #for idx, user in enumerate(df_array): 
        for user_index in users_in_cluster:
            user_df = df_array[user_index-1]  # Get the user's DataFrame from the array
            print(f"User {user_index}") 
                      
            #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
            local_dense_moe_model = m1.build_topk_bilstm_moe_model(X_train[f'user{user_index}'], batch_size, horizon, lstm_units, num_experts, top_k, expert_units, m1)
            local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), metrics=metrics)

            #set local model weight to the weight of the global model
            local_dense_moe_model.set_weights(global_dense_moe_weights)
            
            #Fit local model to local data
            dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
                model=local_dense_moe_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = max_epochs, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=bilstm_moe_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
            )
            #add model weights to list        
            local_dense_moe_weights = local_dense_moe_model.get_weights()
            local_dense_moe_weight_list.append(local_dense_moe_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights_dense_moe = avg_weights_with_noise_fedprox(local_dense_moe_weight_list)
        #update global model 
        global_dense_moe.set_weights(average_weights_dense_moe)
        #Save global models
        global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{bilstm_moe_architecture}/FederatedRound{federated_round+1}")
        print("Saved Global models")


#Evaluation
dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

for cluster_number, users_in_cluster in cluster_users.items():
    print(f"Cluster {cluster_number}:")

    #Get global models weights
    global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{bilstm_moe_architecture}/FederatedRound{federated_rounds}", compile=False)

    #for idx, user in enumerate(df_array): 
    for user_index in users_in_cluster:
        print("User: ", user_index)
        for round in range(3):
            global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{bilstm_moe_architecture}/FederatedRound{federated_rounds}", compile=False)
            dense_moe_model = m1.build_topk_bilstm_moe_model(X_train[f'user{user_index}'], batch_size, horizon, lstm_units, num_experts, top_k, expert_units, m1)
            dense_moe_model.set_weights(global_dense_moe.get_weights())
            
            dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
                model=dense_moe_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = 1, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=bilstm_moe_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006)
            )
            # Add the 'architecture' column from dense_user_results to dense_results
            dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')  

for idx in range(len(df_array)):
    new_row = {
        'architecture': bilstm_moe_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/TopK_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/TopK_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/TopK_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/TopK_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/TopK_BiLSTM_MoE/FederatedRound0\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/TopK_BiLSTM_MoE/FederatedRound3\assets


Saved Global models
Cluster 3:
User 3
User 8
User 9
User 14
User 19
User 27


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/TopK_BiLSTM_MoE/FederatedRound3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/TopK_BiLSTM_MoE/FederatedRound3\assets


Saved Global models
Cluster 0:
User:  1
User:  2
User:  4
User:  5
User:  7
User:  10
User:  11
User:  12
User:  15
User:  16
User:  17
User:  21
User:  22
User:  23
User:  24
User:  25
User:  28
User:  30
Cluster 1:
User:  29
Cluster 2:
User:  6
User:  13
User:  18
User:  20
User:  26
Cluster 3:
User:  3
User:  8
User:  9
User:  14
User:  19
User:  27


In [12]:
dense_results.to_csv(f'evaluations/clustered_federated_learning/{bilstm_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_MoE_e4d8_d16,1.495996,1.413643,0.034502,0.000277,0.185745,0.000746,9.506578e+04,10724.872810,0.132685,0.001910
1,Dense_MoE_e4d8_d16,1.350153,1.270475,0.017793,0.000942,0.133360,0.003506,2.148979e+05,31327.194595,0.074563,0.001257
2,Dense_MoE_e4d8_d16,1.374985,1.293299,0.019011,0.000143,0.137878,0.000519,5.378165e+04,5117.348585,0.073185,0.004528
3,Dense_MoE_e4d8_d16,1.375389,1.294036,0.021595,0.000361,0.146948,0.001226,1.556666e+05,11608.861413,0.063102,0.001889
4,Dense_MoE_e4d8_d16,1.382867,1.301181,0.013548,0.000742,0.116367,0.003170,7.609251e+04,9830.769647,0.070237,0.001908
...,...,...,...,...,...,...,...,...,...,...,...
115,TopK_BiLSTM_MoE,4.228470,4.067098,0.023336,0.000537,0.152753,0.001762,1.282613e+05,12152.387653,0.094571,0.001481
116,TopK_BiLSTM_MoE,4.381140,4.215938,0.017637,0.000514,0.132796,0.001936,2.019347e+05,23894.619120,0.093720,0.003530
117,TopK_BiLSTM_MoE,4.209606,4.047897,0.010369,0.000158,0.101824,0.000775,4.658706e+04,3830.746049,0.044600,0.002256
118,TopK_BiLSTM_MoE,4.176567,4.016528,0.017299,0.000183,0.131526,0.000695,3.683270e+05,45011.451102,0.094126,0.001845
